In [ ]:
# to download the dataset and pretrained model
!gdown --id 1D79dNcLXu6mV1Uueo7EJJXtQm2AO9yUb
# to unzip the file
!unzip dataset.zip

**Caption Testing on pre-trained model**

In [ ]:
# Caption testing on pre-trained model

from pickle import load
from numpy import argmax
from keras_preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model

import IPython.display as display
from PIL import Image

# extract features from the photo
def extract_features(filename):
  # load the model
  model = VGG16()
  # re-structure the model
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  # load the photo
  image = load_img(filename, target_size=(224,224))
  # convert the image pixels to numpy array
  image = img_to_array(image)
  # reshape data for the model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  # prepare the image for the VGG model
  image = preprocess_input(image)
  # get features
  feature = model.predict(image, verbose=0)
  return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
  # seed the generation process
  in_text = 'startseq'
  # iterate over the whole length of the sequence
  for i in range(max_length):
      # integer encode input sequence
      sequence = tokenizer.texts_to_sequences([in_text])[0]
      # pad input
      sequence = pad_sequences([sequence], maxlen=max_length)
      # predict next word
      yhat = model.predict([photo, sequence], verbose=0)
      # convert probability to integer
      yhat = argmax(yhat)
      # map integer to word
      word = word_for_id(yhat, tokenizer)
      # stop if cannot map the word
      if word is None:
          break
      # append as input for generating the next word
      in_text += ' ' + word
      # stop if the end of the sequence is predicted
      if word == 'endseq':
          break
  return in_text

# load the tokenizer
tokenizer = load(open('/content/tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from pre-training)
max_length = 34
# load the pre-trained model
model = load_model('/content/model_4.h5')
# load and prepare the photograph
photo = extract_features('/content/example.jpg')
# display the image
display.display(Image.open('/content/example.jpg'))
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

**Training using smaller dataset and pre-trained model**

In [ ]:
# Feature extraction on training images

from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  # process line by line
  for line in doc.split('\n'):
      # skip empty lines
      if len(line) < 1:
          continue
      # get the image identifier
      identifier = line.split('.')[0]
      dataset.append(identifier)
  return set(dataset)

# extract features from each photo in the directory
def extract_features(directory, dataset):
  # load the model
  model = VGG16()
  # re-structure the model
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  # summarize
  print(model.summary())
  # extract features from each photo
  features = dict()
  for name in dataset:
    # load an image from file
    filename = directory + '/' + name + '.jpg'
    image = load_img(filename, target_size=(224,224))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)
    # get features
    feature = model.predict(image, verbose=0)
    # get image id
    image_id = name.split('.')[0]
    # store feature
    features[image_id] = feature
    print('>%s' % name)
  return features

# extract features from training images
filename = '/content/Flickr8k_text/Flickr_100.trainImages.txt'
train = load_set(filename)
train_features = extract_features('/content/Flickr8k_Dataset', train)
print('Extracted Features of training data: %d' % len(train_features))
# save to file
dump(train_features, open('/content/features_train.pkl', 'wb'))

# extract features from testing images
filename = '/content/Flickr8k_text/Flickr_50.testImages.txt'
test = load_set(filename)
test_features = extract_features('/content/Flickr8k_Dataset', test)
print('Extracted Features of test data: %d' % len(test_features))
# save to file
dump(test_features, open('/content/features_test.pkl', 'wb'))

In [ ]:
# extract desccriptions for images

import string

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# extract descriptions for images
def load_descriptions(doc):
  mapping = dict()
  # process lines
  for line in doc.split('\n'):
      # split line by white space
      tokens = line.split()
      if len(line) < 2:
          continue
      # take the first token as the image id, the rest as the description
      image_id, image_desc = tokens[0], tokens[1:]
      # remove filename from image id
      image_id = image_id.split('.')[0]
      # convert description tokens back to string
      image_desc = ' '.join(image_desc)
      # create the list if needed:
      if image_id not in mapping:
          mapping[image_id] = list()
      # store description
      mapping[image_id].append(image_desc)
  return mapping

def clean_descriptions(descriptions):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in descriptions.items():
      for i in range(len(desc_list)):
          desc = desc_list[i]
          # tokenize
          desc = desc.split()
          # convert to lower case
          desc = [word.lower() for word in desc]
          # remove punctuation from each token
          desc = [w.translate(table) for w in desc]
          # remove hanging 's' and 'a'
          desc = [word for word in desc if len(word)>1]
          # remove tokens with numbers
          desc = [word for word in desc if word.isalpha()]
          # store as string
          desc_list[i] = ' '.join(desc)
            
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
  # build a list of all description strings
  all_desc = set()
  for key in descriptions.keys():
      [all_desc.update(d.split()) for d in descriptions[key]]
  return all_desc

# save descriptions to file with one per line
def save_descriptions(descriptions, filename):
  lines = list()
  for key, desc_list in descriptions.items():
      for desc in desc_list:
          lines.append(key + ' ' + desc)
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()
    
filename = '/content/Flickr8k_text/Flickr8k.token.txt'
# load descriptions
doc = load_doc(filename)
# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, '/content/descriptions.txt')

In [ ]:
# Train the model

from numpy import array
from pickle import load
from numpy import argmax
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.models import load_model

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  # process line by line
  for line in doc.split('\n'):
      # skip empty lines
      if len(line) < 1:
          continue
      # get the image identifier
      identifier = line.split('.')[0]
      dataset.append(identifier)
  return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
      # split line by white space
      tokens = line.split()
      # split id from description
      image_id, image_desc = tokens[0], tokens[1:]
      # skip images not in the set
      if image_id in dataset:
          # create list
          if image_id not in descriptions:
              descriptions[image_id] = list()
          # wrap description in tokens
          desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
          # store
          descriptions[image_id].append(desc)
  return descriptions

# load photo features
def load_photo_features(filename, dataset):
  # load all features
  all_features = load(open(filename, 'rb'))
  # filter features
  features = {k: all_features[k] for k in dataset}
  return features

# create sequences of images, input sequences and output word for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
  X1, X2, y = list(), list(), list()
  # walk through each description for the image
  for desc in desc_list:
      # encode the sequence
      seq = tokenizer.texts_to_sequences([desc])[0]
      # split one sequence into multiple X,y pairs
      for i in range(1, len(seq)):
          # split into input and output pair
          in_seq, out_seq = seq[:i], seq[i]
          # pad input sequence
          in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
          # encode output sequence
          out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
          # store
          X1.append(photo)
          X2.append(in_seq)
          y.append(out_seq)
  return array(X1), array(X2), array(y)

# define captioning model
def define_model():
  model = load_model('/content/model_4.h5')
  # summarize model
  print(model.summary())
  return model

# intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
  # loop forever over images
  while 1:
      for key, desc_list in descriptions.items():
          # retrieve the photo feature
          photo = photos[key][0]
          in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
          yield [in_img, in_seq], out_word

# define train dataset

# load training dataset (100)
filename = '/content/Flickr8k_text/Flickr_100.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/content/features_train.pkl', train)
print('Photos: train=%d' % len(train_features))
# load the tokenizer
tokenizer = load(open('/content/tokenizer.pkl', 'rb'))
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# pre-define the max sequence length (from pre-training)
max_length = 34
print('Description Length: %d' % max_length)

# define the model
model = define_model()
# train the model, run epochs and save model after each epoch
epochs = 5
steps = len(train_descriptions)
for i in range(epochs):
  # create data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
  # fit for one epoch
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
  # save model
  model.save('/content/lab_model_' + str(i) + '.h5')

In [ ]:
# evaluating the trained model

from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

import IPython.display as display
from PIL import Image

# load doc into memory
def load_doc(filename):
  # open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  doc = load_doc(filename)
  dataset = list()
  # process line by line
  for line in doc.split('\n'):
      # skip empty lines
      if len(line) < 1:
          continue
      # get the image identifier
      identifier = line.split('.')[0]
      dataset.append(identifier)
  return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
      # split line by white space
      tokens = line.split()
      # split id from description
      image_id, image_desc = tokens[0], tokens[1:]
      # skip images not in the set
      if image_id in dataset:
          # create list
          if image_id not in descriptions:
              descriptions[image_id] = list()
          # wrap description in tokens
          desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
          # store
          descriptions[image_id].append(desc)
  return descriptions

# load photo features
def load_photo_features(filename, dataset):
  # load all features
  all_features = load(open(filename, 'rb'))
  # filter features
  features = {k: all_features[k] for k in dataset}
  return features

# map an integer to a word
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
  # seed the generation process
  in_text = 'startseq'
  # iterate over the whole length of the sequence
  for i in range(max_length):
      # integer encode input sequence
      sequence = tokenizer.texts_to_sequences([in_text])[0]
      # pad input
      sequence = pad_sequences([sequence], maxlen=max_length)
      # predict next word
      yhat = model.predict([photo, sequence], verbose=0)
      # convert probability to integer
      yhat = argmax(yhat)
      # map integer to word
      word = word_for_id(yhat, tokenizer)
      # stop if cannot map the word
      if word is None:
          break
      # append as input for generating the next word
      in_text += ' ' + word
      # stop if the end of the sequence is predicted
      if word == 'endseq':
          break
  return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
  actual, predicted = list(), list()
  # step over the whole set
  for key, desc_list in descriptions.items():
      # generate description
      yhat = generate_desc(model, tokenizer, photos[key], max_length)
      # store actual and predicted
      references = [d.split() for d in desc_list]
      actual.append(references)
      predicted.append(yhat.split())
  # calculate BLEU score
  print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
  print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
  print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
  print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
  
  # display the predicted caption of the last photo
  display.display(Image.open('/content/Flickr8k_Dataset/' + key +'.jpg'))
  print(yhat)
    
# prepare test set

# load test set
filename = '/content/Flickr8k_text/Flickr_50.testImages.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('/content/descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/content/features_test.pkl', test)
print('Photos: test=%d' % len(test_features))
# load the tokenizer
tokenizer = load(open('/content/tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from pre-training)
max_length = 34

# load the model
filename = '/content/lab_model_4.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)